# **ResNet50 Model - Fashion MNIST**

## **Data Downloading and Preprocessing**

### **Imports**

In [1]:
# Imports
import torch
import torchvision
import torchvision.transforms as transforms

### **Transformations**

In [2]:
# Define data transformations
transform = transforms.Compose([
    # Convert images to 3-channel grayscale (RGB)
    transforms.Grayscale(num_output_channels = 3),

    # Resize the image to 224x224 pixels
    transforms.Resize((224, 224)),

    # Convert the image to a PyTorch tensor
    transforms.ToTensor(),

    # Normalize the image
    transforms.Normalize(mean = [0.5, 0.5, 0.5], std = [0.5, 0.5, 0.5])
])

### **Download and Load Data**

In [3]:
# Download and load the training data
trainset = torchvision.datasets.FashionMNIST(root = './data', train = True, download = True, transform = transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size = 64, shuffle = True, num_workers = 2)

# Download and load the test data
testset = torchvision.datasets.FashionMNIST(root = './data', train = False, download = True, transform = transform)
testloader = torch.utils.data.DataLoader(testset, batch_size = 64, shuffle = False, num_workers = 2)

## **Model Loading and Training**

### **Impots**

In [4]:
# Imports
import torch.nn as nn
import torch.optim as optim
from torchvision.models import resnet50
from torchvision.models import ResNet50_Weights
from tqdm import tqdm
import time

try:
    import torchmetrics
except:
    !pip install torchmetrics --quiet
    import torchmetrics

### **Device Agnostic Setting**

In [5]:
# Check if a GPU is available, otherwise use CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

### **ResNet50 Model**

In [6]:
# Load the ResNet50 model with pretrained weights
model = resnet50(weights = ResNet50_Weights.IMAGENET1K_V1)

In [7]:
# Modify the fully connected layer to match the number of classes in Fashion MNIST (10 classes)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 10)

In [8]:
# Move the model to the available device (GPU or CPU)
model = model.to(device)

### **Loss, Optimizer and Accuracy**

In [9]:
# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = 1e-3)
accuracy = torchmetrics.Accuracy(task = "multiclass", num_classes = 10).to(device)

### **Training the Model**

In [10]:
# Store the start time
start_time = time.time()

# Initialize number of epochs
num_epochs = 5

# Loop for epochs
for epoch in range(num_epochs):
    # Set the model to train mode
    model.train()

    # Initialize running loss and accuracy
    running_loss, running_acc = 0.0, 0.0

    # Traverse over the batches of data
    for inputs, labels in tqdm(trainloader, desc=f"Epoch {epoch + 1}/{num_epochs}", leave=False):
        # Move the data to device
        inputs, labels = inputs.to(device), labels.to(device)

        # Optimizer zero grad
        optimizer.zero_grad()

        # Get the outputs on the inputs
        outputs = model(inputs)

        # Get the loss
        loss = criterion(outputs, labels)

        # Backpropagate the loss
        loss.backward()

        # Update the optimizer parameters
        optimizer.step()

        # Update the running loss and accuracy
        running_loss += loss.item() / len(trainloader)
        running_acc += accuracy(outputs.argmax(dim=1), labels) / len(trainloader)

    # Print the information
    print(f"Epoch {epoch + 1}/{num_epochs} | Train Loss: {running_loss:.4f} | Train Accuracy: {running_acc * 100:.4f}%")

# Get the end time
end_time = time.time()

# Print the total training time
print(f"Training Time: {end_time - start_time} seconds")

Epoch 1/5 | Train Loss: 0.3626 | Train Accuracy: 86.9037%


Epoch 2/5 | Train Loss: 0.2489 | Train Accuracy: 91.0598%


Epoch 3/5 | Train Loss: 0.2111 | Train Accuracy: 92.2924%


Epoch 4/5 | Train Loss: 0.1868 | Train Accuracy: 93.2985%


Epoch 5/5 | Train Loss: 0.1675 | Train Accuracy: 93.9182%
Training Time: 3202.880265712738 seconds


## **Model Testing and Evaluation**

In [11]:
# Set model to eval mode
model.eval()

# Lists to store the predictions and labels
predictions, true_labels = [], []

# Store the start time
start_time = time.time()

# Turn on inference context manager
with torch.inference_mode():
    # Loop over testing batches
    for inputs, labels in tqdm(testloader, desc = "Testing", leave = False):
        # Move the data to device
        inputs, labels = inputs.to(device), labels.to(device)

        # Get the outputs
        outputs = model(inputs)

        # Get the predicted label
        predicted = outputs.argmax(dim = 1)

        # Update the lists
        predictions.extend(predicted.cpu().numpy())
        true_labels.extend(labels.cpu().numpy())

# Get the end time
end_time = time.time()

# Print the total training time
print(f"Prediction Time: {end_time - start_time} seconds")

Prediction Time: 33.99074840545654 seconds


## **Evaluation with Classification Report**

In [12]:
# Import classification report
from sklearn.metrics import classification_report

# Generate and print the classification report
print("Classification Report:")
print(classification_report(true_labels, predictions, target_names = trainset.classes))

Classification Report:
              precision    recall  f1-score   support

 T-shirt/top       0.89      0.85      0.87      1000
     Trouser       0.99      0.99      0.99      1000
    Pullover       0.89      0.91      0.90      1000
       Dress       0.92      0.93      0.93      1000
        Coat       0.91      0.86      0.88      1000
      Sandal       0.99      0.98      0.98      1000
       Shirt       0.75      0.82      0.78      1000
     Sneaker       0.97      0.95      0.96      1000
         Bag       1.00      0.97      0.99      1000
  Ankle boot       0.95      0.98      0.97      1000

    accuracy                           0.93     10000
   macro avg       0.93      0.93      0.93     10000
weighted avg       0.93      0.93      0.93     10000



## **Saving Model**

In [13]:
# Mount google drive
from google.colab import drive
drive.mount("./Drive", force_remount = True)

Mounted at ./Drive


In [14]:
# Path to handle paths to files / folders
from pathlib import Path

# Create models directory
MODEL_PATH = Path("./Drive/MyDrive/PyTorch Course/models")
MODEL_PATH.mkdir(parents = True, exist_ok = True)

# Create model save path
MODEL_NAME = "03_pytorch_fashion_mnist_resnet50.pth"
MODEL_SAVE_PATH = MODEL_PATH / MODEL_NAME

# Save the model state dict
print(f"Saving Model to: {MODEL_SAVE_PATH}")
torch.save(obj = model.state_dict(), f = MODEL_SAVE_PATH)

Saving Model to: Drive/MyDrive/PyTorch Course/models/03_pytorch_fashion_mnist_resnet50.pth


In [15]:
# Check the saved file path
!ls -l "./Drive/MyDrive/PyTorch Course/models/03_pytorch_fashion_mnist_resnet50.pth"

-rw------- 1 root root 94439733 Jul 21 12:02 './Drive/MyDrive/PyTorch Course/models/03_pytorch_fashion_mnist_resnet50.pth'


## **Load Model**

### **Initialize Model**

In [16]:
# Load the ResNet50 model
loaded_model = resnet50(weights = ResNet50_Weights.IMAGENET1K_V1)

# Modify the fully connected layer to match the number of classes in Fashion MNIST (10 classes)
num_ftrs = loaded_model.fc.in_features
loaded_model.fc = nn.Linear(num_ftrs, 10)

# Move the model to the available device (GPU or CPU)
loaded_model = loaded_model.to(device)

# Load the state_dict of our saved model (this will update the new instance of our model with trained weights)
loaded_model.load_state_dict(torch.load(f = MODEL_SAVE_PATH))

<All keys matched successfully>

### **Generate Outputs**

In [17]:
# Set model to eval mode
model.eval()

# Lists to store the predictions and labels
predictions, true_labels = [], []

# Store the start time
start_time = time.time()

# Turn on inference context manager
with torch.inference_mode():
    # Loop over testing batches
    for inputs, labels in tqdm(testloader, desc = "Testing", leave = False):
        # Move the data to device
        inputs, labels = inputs.to(device), labels.to(device)

        # Get the outputs
        outputs = model(inputs)

        # Get the predicted label
        predicted = outputs.argmax(dim = 1)

        # Update the lists
        predictions.extend(predicted.cpu().numpy())
        true_labels.extend(labels.cpu().numpy())

# Get the end time
end_time = time.time()

# Print the total training time
print(f"Prediction Time: {end_time - start_time} seconds")

Prediction Time: 34.53281021118164 seconds


In [18]:
# Import classification report
from sklearn.metrics import classification_report

# Generate and print the classification report
print("Classification Report:")
print(classification_report(true_labels, predictions, target_names = trainset.classes))

Classification Report:
              precision    recall  f1-score   support

 T-shirt/top       0.89      0.85      0.87      1000
     Trouser       0.99      0.99      0.99      1000
    Pullover       0.89      0.91      0.90      1000
       Dress       0.92      0.93      0.93      1000
        Coat       0.91      0.86      0.88      1000
      Sandal       0.99      0.98      0.98      1000
       Shirt       0.75      0.82      0.78      1000
     Sneaker       0.97      0.95      0.96      1000
         Bag       1.00      0.97      0.99      1000
  Ankle boot       0.95      0.98      0.97      1000

    accuracy                           0.93     10000
   macro avg       0.93      0.93      0.93     10000
weighted avg       0.93      0.93      0.93     10000

